#Uploading the data

In [ ]:
from google.colab import drive

# Mount Google Drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import pandas as pd

# Read the dataset from Google Drive, specifying the encoding as 'latin1' or 'ISO-8859-1'
dataset = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/data/TSX_Companies_yf.csv', encoding='latin1')

# Drop rows with any missing values (NaN) and reset the index
dataset = dataset.dropna().reset_index(drop=True)

dataset

#yFinance

In [ ]:
dataset_test = dataset[:10]

In [ ]:
dataset_test

In [ ]:
import pandas as pd
import yfinance as yf

#DATABASE OF ALL TICKERS AND SOME ATTRIBUTES.

# Create an empty DataFrame to store the results
TSX_Data = pd.DataFrame(columns=["Ticker", "Company", "opcashflw_2023", "opcashflw_2022", "opcashflw_2021","total_assets_2023","total_assets_2022","total_assets_2021", "equity_2023", "equity_2022", "equity_2021", "liability_2023", "liability_2022", "liability_2021"])

# Iterate through each row in the dataset
for index, row in dataset.iterrows():
    try:
        print(f"Processing row {index + 1}...")
        # Fetch the company information
        ticker = row['Ticker']
        company = yf.Ticker(ticker)

        company_name = company.info.get('longName')
        print(f"Company: {company_name} Ticker: {ticker}")

        #Total assets
        total_assets = None
        if not company.balance_sheet.empty and 'Total Assets' in company.balance_sheet.index:
            total_assets = company.balance_sheet.loc['Total Assets']
        else:
            total_assets = None

        if total_assets is not None:
            keys = list(total_assets.keys())
            total_assets_2023 = total_assets.get(keys[0], None) if len(keys) > 0 else None
            total_assets_2022 = total_assets.get(keys[1], None) if len(keys) > 1 else None
            total_assets_2021 = total_assets.get(keys[2], None) if len(keys) > 2 else None
        else:
            total_assets_2023 = total_assets_2022 = total_assets_2021 = "N/A"

        #Equity
        equity = None
        if not company.balance_sheet.empty and 'Total Equity Gross Minority Interest' in company.balance_sheet.index:
            equity = company.balance_sheet.loc['Total Equity Gross Minority Interest']
        else:
            equity = None

        if equity is not None:
            keys = list(equity.keys())
            equity_2023 = equity.get(keys[0], None) if len(keys) > 0 else None
            equity_2022 = equity.get(keys[1], None) if len(keys) > 1 else None
            equity_2021 = equity.get(keys[2], None) if len(keys) > 2 else None
        else:
            equity_2023 = equity_2022 = equity_2021 = "N/A"

        #Liability
        liability = None
        if not company.balance_sheet.empty and 'Total Equity Gross Minority Interest' in company.balance_sheet.index:
            liability = company.balance_sheet.loc['Total Equity Gross Minority Interest']
        else:
            liability = None

        if liability is not None:
            keys = list(liability.keys())
            liability_2023 = liability.get(keys[0], None) if len(keys) > 0 else None
            liability_2022 = liability.get(keys[1], None) if len(keys) > 1 else None
            liability_2021 = liability.get(keys[2], None) if len(keys) > 2 else None
        else:
            liability_2023 = liability_2022 = liability_2021 = "N/A"

        #Operating cashflow
        opcashflw = None

        if not company.cash_flow.empty and 'Operating Cash Flow' in company.cash_flow.index:
            opcashflw = company.cash_flow.loc['Operating Cash Flow']
        else:
            opcashflw = None

        if opcashflw is not None:
            keys = list(opcashflw.keys())
            opcashflw_2023 = opcashflw.get(keys[0], None) if len(keys) > 0 else None
            opcashflw_2022 = opcashflw.get(keys[1], None) if len(keys) > 1 else None
            opcashflw_2021 = opcashflw.get(keys[2], None) if len(keys) > 2 else None
        else:
            opcashflw_2023 = opcashflw_2022 = opcashflw_2021 = "N/A"

        # Append the symbol, sector, and EBITDA to the DataFrame
        TSX_Data.loc[len(TSX_Data)] = [ticker, company_name, opcashflw_2023, opcashflw_2022, opcashflw_2021,total_assets_2023, total_assets_2022, total_assets_2021, equity_2023, equity_2022, equity_2021, liability_2023, liability_2022, liability_2021,]
    except Exception as e:
            # If an error occurs (e.g., ticker not found), append None for the sector and revenue
            TSX_Data.loc[len(TSX_Data)] = [ticker, company_name, None, None, None, ]
            print(f"Error processing row {index + 1}: {e}")


In [ ]:
TSX_Data.dropna().reset_index(drop=True)

In [ ]:
# Merge datasets based on the 'Ticker' column
merged_dataset = pd.merge(dataset, TSX_Data, on='Ticker', how='left')

# Drop the "Company_y" column
merged_dataset.drop(columns=["Company_y"], inplace=True)

# Rename the "Company_x" column to "Company"
merged_dataset.rename(columns={"Company_x": "Company"}, inplace=True)

In [ ]:
merged_dataset

,Ticker,Company,Sector,Revenue_2022,Revenue_2021,Revenue_2020,opcashflw_2023,opcashflw_2022,opcashflw_2021,total_assets_2023,total_assets_2022,total_assets_2021,equity_2023,equity_2022,equity_2021,liability_2023,liability_2022,liability_2021
0,AAB.TO,Aberdeen International Inc.,Financial Services,-8.762676e+06,9.139536e+06,5.628492e+06,-2609555.0,689969.0,-1139027.0,3.041472e+07,4.309594e+07,34234649.0,2.878224e+07,4.076537e+07,32377884.0,2.878224e+07,4.076537e+07,32377884.0
1,ADN.TO,Acadian Timber Corp.,Basic Materials,9.047300e+07,9.572900e+07,9.103100e+07,15235000.0,16229000.0,20617000.0,5.478360e+08,5.166420e+08,513398000.0,3.037230e+08,2.914850e+08,293240000.0,3.037230e+08,2.914850e+08,293240000.0
2,ACD.TO,Accord Financial Corp.,Financial Services,4.340363e+07,4.759376e+07,3.390492e+07,31507257.0,-101646734.0,23370653.0,4.917612e+08,5.201087e+08,384912638.0,1.066121e+08,1.039596e+08,93759124.0,1.066121e+08,1.039596e+08,93759124.0
3,ARA.TO,Aclara Resources Inc.,Basic Materials,0.000000e+00,0.000000e+00,0.000000e+00,-8441000.0,-4024000.0,2511000.0,1.568170e+08,1.648170e+08,75162000.0,1.531870e+08,1.617670e+08,69130000.0,1.531870e+08,1.617670e+08,69130000.0
4,ADCO.TO,Adcore Inc.,Technology,1.969300e+07,2.796600e+07,1.699700e+07,-2547000.0,-1436000.0,5524000.0,1.454300e+07,1.728400e+07,12736000.0,9.734000e+06,1.219500e+07,6885000.0,9.734000e+06,1.219500e+07,6885000.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
667,WSP.TO,WSP Global Inc.,Industrials,1.193290e+10,1.027910e+10,8.803900e+09,814800000.0,1060100000.0,1125100000.0,1.484170e+10,1.125040e+10,8837400000.0,6.009100e+09,4.665200e+09,4081400000.0,6.009100e+09,4.665200e+09,4081400000.0
668,XTG.TO,Xtra-Gold Resources Corp.,Basic Materials,0.000000e+00,0.000000e+00,0.000000e+00,1528415.0,670948.0,505415.0,1.188101e+07,1.075803e+07,9340942.0,1.047433e+07,9.635548e+06,8914123.0,1.047433e+07,9.635548e+06,8914123.0
669,XTRA.TO,Xtract One Technologies Inc.,Technology,4.111244e+06,3.619214e+06,1.081975e+06,-13535322.0,-9271441.0,-11694837.0,1.899883e+07,1.805818e+07,49398424.0,1.474290e+07,1.454518e+07,46838346.0,1.474290e+07,1.454518e+07,46838346.0
670,Y.TO,Yellow Pages Limited,Communication Services,2.682780e+08,2.876460e+08,3.335380e+08,49500000.0,104579000.0,126998000.0,2.072980e+08,3.052290e+08,366379000.0,6.578100e+07,1.161310e+08,29301000.0,6.578100e+07,1.161310e+08,29301000.0


In [ ]:
# Save the shrunk dataset to a new CSV file
merged_dataset.to_csv('/content/drive/MyDrive/##File Path##/TSX_667_yfData.csv', index=False, sep=',')

# Provide a download link for the file
from google.colab import files
files.download('/content/drive/MyDrive/##File Path##/TSX_667_yfData.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

#Data exploration

In [ ]:
import pandas as pd

# Read the dataset from Google Drive, specifying the encoding as 'latin1' or 'ISO-8859-1'
graphicData = pd.read_csv('/content/drive/MyDrive/##File Path##/TSX_667_yfData.csv', encoding='latin1')

# Drop rows with any missing values (NaN) and reset the index
graphicData = graphicData.dropna().reset_index(drop=True)

graphicData

,Ticker,Company,Sector,Revenue_2022,Revenue_2021,Revenue_2020,opcashflw_2023,opcashflw_2022,opcashflw_2021,total_assets_2023,total_assets_2022,total_assets_2021,equity_2023,equity_2022,equity_2021,liability_2023,liability_2022,liability_2021
0,AAB.TO,Aberdeen International Inc.,Financial Services,-8.762676e+06,9.139536e+06,5.628492e+06,-2609555.0,6.899690e+05,-1.139027e+06,3.041472e+07,4.309594e+07,3.423465e+07,2.878224e+07,4.076537e+07,3.237788e+07,2.878224e+07,4.076537e+07,3.237788e+07
1,ADN.TO,Acadian Timber Corp.,Basic Materials,9.047300e+07,9.572900e+07,9.103100e+07,15235000.0,1.622900e+07,2.061700e+07,5.478360e+08,5.166420e+08,5.133980e+08,3.037230e+08,2.914850e+08,2.932400e+08,3.037230e+08,2.914850e+08,2.932400e+08
2,ACD.TO,Accord Financial Corp.,Financial Services,4.340363e+07,4.759376e+07,3.390492e+07,31507257.0,-1.016467e+08,2.337065e+07,4.917612e+08,5.201087e+08,3.849126e+08,1.066121e+08,1.039596e+08,9.375912e+07,1.066121e+08,1.039596e+08,9.375912e+07
3,ARA.TO,Aclara Resources Inc.,Basic Materials,0.000000e+00,0.000000e+00,0.000000e+00,-8441000.0,-4.024000e+06,2.511000e+06,1.568170e+08,1.648170e+08,7.516200e+07,1.531870e+08,1.617670e+08,6.913000e+07,1.531870e+08,1.617670e+08,6.913000e+07
4,ADCO.TO,Adcore Inc.,Technology,1.969300e+07,2.796600e+07,1.699700e+07,-2547000.0,-1.436000e+06,5.524000e+06,1.454300e+07,1.728400e+07,1.273600e+07,9.734000e+06,1.219500e+07,6.885000e+06,9.734000e+06,1.219500e+07,6.885000e+06
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
662,WSP.TO,WSP Global Inc.,Industrials,1.193290e+10,1.027910e+10,8.803900e+09,814800000.0,1.060100e+09,1.125100e+09,1.484170e+10,1.125040e+10,8.837400e+09,6.009100e+09,4.665200e+09,4.081400e+09,6.009100e+09,4.665200e+09,4.081400e+09
663,XTG.TO,Xtra-Gold Resources Corp.,Basic Materials,0.000000e+00,0.000000e+00,0.000000e+00,1528415.0,6.709480e+05,5.054150e+05,1.188101e+07,1.075803e+07,9.340942e+06,1.047433e+07,9.635548e+06,8.914123e+06,1.047433e+07,9.635548e+06,8.914123e+06
664,XTRA.TO,Xtract One Technologies Inc.,Technology,4.111244e+06,3.619214e+06,1.081975e+06,-13535322.0,-9.271441e+06,-1.169484e+07,1.899883e+07,1.805818e+07,4.939842e+07,1.474290e+07,1.454518e+07,4.683835e+07,1.474290e+07,1.454518e+07,4.683835e+07
665,Y.TO,Yellow Pages Limited,Communication Services,2.682780e+08,2.876460e+08,3.335380e+08,49500000.0,1.045790e+08,1.269980e+08,2.072980e+08,3.052290e+08,3.663790e+08,6.578100e+07,1.161310e+08,2.930100e+07,6.578100e+07,1.161310e+08,2.930100e+07


In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px

testingdata = graphicData

# Check for missing values
print(testingdata.isnull().sum())

Ticker               0
Company              0
Sector               0
Revenue_2022         0
Revenue_2021         0
Revenue_2020         0
opcashflw_2023       0
opcashflw_2022       0
opcashflw_2021       0
total_assets_2023    0
total_assets_2022    0
total_assets_2021    0
equity_2023          0
equity_2022          0
equity_2021          0
liability_2023       0
liability_2022       0
liability_2021       0
dtype: int64


In [ ]:
# Count the number of companies in each sector
testingdata["Sector"].value_counts()

Basic Materials           189
Energy                     98
Financial Services         84
Industrials                61
Healthcare                 53
Technology                 46
Consumer Cyclical          45
Consumer Defensive         30
Utilities                  21
Real Estate                21
Communication Services     19
Name: Sector, dtype: int64

#Dashboard Creation

In [ ]:
!pip install dash
!pip install pyngrok


In [ ]:
# Define the style as variables
title_style = {
    'textAlign': 'center',
    'marginBottom': '10px',
    'fontFamily': 'Arial, sans-serif',
    'fontSize': '36px',
    'fontWeight': 'bold',
    'color': '#F6F6F6',  # White text color
    'letterSpacing': '2px',
    'borderBottom': '2px solid #4CAF50',  # Accent color
    'paddingBottom': '10px'  # Adding padding to the bottom
}

subtitle_style = {
    'textAlign': 'center',
    'fontFamily': 'Arial, sans-serif',
    'fontSize': '20px',
    'fontWeight': 'bold',
    'color': '#F6F6F6',  # White text color
    'marginTop': '5px',  # Adding margin to the top
    'marginBottom': '15px'  # Adding margin to the bottom
}

paragraph_style = {
    'textAlign': 'left',
    'fontFamily': 'Arial, sans-serif',
    'fontSize': '16px',
    'color': '#F6F6F6',  # White text color
    'lineHeight': '1.5',  # Adding line height for readability
    'marginBottom': '20px',  # Adding margin to the bottom
    'display': 'inline-block',
    'width': '18%',
    'verticalAlign': 'top',
    'backgroundColor': '#607D8B',  # Primary color
    'padding': '10px',
    'boxShadow': '0px 4px 6px rgba(0, 0, 0, 0.1)',  # Adding a subtle shadow
    'margin-right': '10px'
}



button_style = {
    'marginTop': '10px',
    'textAlign': 'center',
    'backgroundColor': '#607D8B',  # Primary color
    'color': '#F6F6F6',  # White text color
    'fontFamily': 'Arial, sans-serif',
    'fontSize': '16px',
    'fontWeight': 'bold',
    'padding': '5px 10px',  # Padding for better spacing
    'borderRadius': '5px',  # Rounded corners
    'border': 'none',  # No border
    'cursor': 'pointer',  # Cursor style on hover
    'boxShadow': '0px 4px 6px rgba(0, 0, 0, 0.1)',  # Adding a subtle shadow
    'transition': 'background-color 0.3s ease-in-out'  # Smooth transition on hover
}

button_style2 = {
    'marginTop': '10px',
    'textAlign': 'center',
    'backgroundColor': '#455A64',  # Secondary color
    'color': '#F6F6F6',  # White text color
    'fontFamily': 'Arial, sans-serif',
    'fontSize': '16px',
    'fontWeight': 'bold',
    'padding': '5px 10px',  # Padding for better spacing
    'borderRadius': '5px',  # Rounded corners
    'border': 'none',  # No border
    'cursor': 'pointer',  # Cursor style on hover
    'margin-right': '10px',
    'boxShadow': '0px 4px 6px rgba(0, 0, 0, 0.1)',  # Adding a subtle shadow
    'transition': 'background-color 0.3s ease-in-out'  # Smooth transition on hover
}



In [182]:
#Create the dashboard
import pandas as pd
import plotly.express as px
import dash
import dash_core_components as dcc
import dash_html_components as html
from dash.dependencies import Input, Output
import plotly.graph_objects as go
import plotly.colors as pc
import plotly.express as px

testingdata = graphicData

# Create Dash app
app = dash.Dash(__name__)

# Define unique sectors
unique_sectors = testingdata["Sector"].unique()

# Create layout for app
def create_layout():
    return html.Div(style={'backgroundColor': '#263238', 'padding-top':'2px'}, children=[
        # Title div
        html.Div([
            html.H1("Market Analysis Dashboard", style=title_style)
        ]),

       # Sector buttons div
        html.Div([
            html.Label("Select Sector:", style=subtitle_style),
            html.Div([
                html.Button(sector, id=f"button-{sector}", n_clicks=0, className='sector-button', style=button_style2)
                for sector in unique_sectors
            ], style={'display': 'flex', 'flexWrap': 'wrap', 'justifyContent': 'center'}),
            # Add button to select all sectors
            html.Button('All Sectors', id='button-all', n_clicks=0, className='sector-button', style=button_style)
        ], style={'textAlign': 'center', 'marginBottom': '20px'}),



       html.Div([
            # Industry metrics div (first column)
            html.Div(id='industry-metrics', style=paragraph_style),

            # Bar chart div (second column)
            html.Div([
                dcc.Graph(
                    id='bar-chart-revenue',
                    style={'width': '100%', 'backgroundColor': '#263238', 'borderRadius': '10px', 'boxShadow': '0px 0px 10px rgba(0, 0, 0, 0.1)'},
                    config={'displayModeBar': False}  # Hides the plotly interactive menu
                )
            ], style={'display': 'inline-block', 'width': '28%', 'padding-right': '10px'}),

            # Graphics div
            html.Div([
                # Placeholder for first graphic: Scatter plot of revenue vs cash flow
                dcc.Graph(id='scatter-plot', style={'width': '100%', 'backgroundColor': '#263238', 'borderRadius': '10px', 'boxShadow': '0px 0px 10px rgba(0, 0, 0, 0.1)'}, config={'displayModeBar': False} ),
            ], style={'display': 'inline-block', 'width': '48%'})
        ], style={'width': '90%', 'margin': '0 auto'})  # Centering the content horizontally with fixed width
    ])

# Define callback to update graphs based on selected sector
@app.callback(
    [Output('scatter-plot', 'figure'),
     Output('bar-chart-revenue', 'figure'),
     Output('industry-metrics', 'children')],
    [Input('button-all', 'n_clicks')] +
    [Input(f"button-{sector}", 'n_clicks') for sector in unique_sectors]
)

def update_graph(n_clicks_all, *args):
    ctx = dash.callback_context

    # Determine selected sector
    if not any(args[:-1]):
        selected_sector = 'ALL' # Default to the first sector
    else:
        triggered_id = ctx.triggered[0]['prop_id'].split('.')[0]
        if triggered_id == 'button-all':
            selected_sector = 'ALL'
        else:
            selected_sector = triggered_id.split('-')[1]

    # Filter data based on selected sector
    if selected_sector == 'ALL':
        filtered_data = testingdata.copy()
    else:
        filtered_data = testingdata[testingdata['Sector'] == selected_sector]



    # Create the scatter plot with updated marker colors
    fig1 = px.scatter(filtered_data, x='Revenue_2022', y='opcashflw_2022', color='Company', title='Revenue vs Cash Flow')


    # Update layout to enhance visibility
    fig1.update_layout(
        plot_bgcolor='#33444B',  # Set the background color of the plot area
        paper_bgcolor='#455A64',  # Set the background color of the entire plot
        font_color='#F6F6F6',  # Set the text color
        title_font_color='#F6F6F6',  # Set the title text color
        legend_bgcolor='rgba(0, 0, 0, 0)',  # Set legend background color to transparent
        legend_font_color='#F6F6F6',  # Set legend text color
        margin=dict(l=60, r=40, t=60, b=40),  # Adjust margins for better spacing
        hovermode='closest',  # Set hover mode
        xaxis_title='Revenue 2022',
        yaxis_title='Cashflow 2022',
    )


    # Calculate average revenue for each year across all companies
    average_revenue_2020 = filtered_data['Revenue_2020'].sum()
    average_revenue_2021 = filtered_data['Revenue_2021'].sum()
    average_revenue_2022 = filtered_data['Revenue_2022'].sum()

    # Create a bar chart with average revenue for each year
    fig2 = go.Figure(go.Bar(
        x=['2020', '2021', '2022'],
        y=[average_revenue_2020, average_revenue_2021, average_revenue_2022],
        marker_color=['#607D8B', '#455A64', '#B0BEC5']
    ))

     # Add a line for total revenue
    fig2.add_trace(go.Scatter(
        x=['2020', '2021', '2022'],
        y=[average_revenue_2020, average_revenue_2021, average_revenue_2022],
        mode='lines+markers',
        name='Trend',
        line=dict(color='#4CAF50', width=2, dash='dash')  # Hover color
    ))

    # Update layout
    fig2.update_layout(
        title='Total Revenue Year Over Year',
        xaxis_title='Year',
        yaxis_title='Total Revenue ($)',
        hovermode='x',
        plot_bgcolor='#33444B',  # Set the background color of the plot area
        paper_bgcolor='#455A64',  # Set the background color of the entire plot
        font_color='#F6F6F6',  # Set the text color
        title_font_color='#F6F6F6',  # Set the title text color
        legend_bgcolor='rgba(0, 0, 0, 0)',  # Set legend background color to transparent
        legend_font_color='#F6F6F6'
    )
    # Calculate industry metrics in thousands
    total_revenue_2022 = filtered_data['Revenue_2022'].mean() / 1000
    total_revenue_2021 = filtered_data['Revenue_2021'].mean() / 1000
    total_revenue_2020 = filtered_data['Revenue_2020'].mean() / 1000
    total_companies = len(filtered_data['Company'].unique())

    # Format industry metrics content with values expressed in thousands
    industry_metrics_title = f"Industry Metrics: {selected_sector}"
    industry_metrics_content = html.Div(style={'fontSize': '18px'}, children=[
        html.P(industry_metrics_title),
        html.P(f"Avg Rev 2022: ${total_revenue_2022:.2f}K"),
        html.P(f"Avg Rev 2021: ${total_revenue_2021:.2f}K"),
        html.P(f"Avg Rev 2020: ${total_revenue_2020:.2f}K"),
        html.P(f"Total Companies: {total_companies}")
    ])

    return fig1, fig2, industry_metrics_content

# Run app
if __name__ == '__main__':
    app.layout = create_layout()  # Set layout
    app.run_server(debug=True, use_ngrok=True)

# Expect a little lag time when changing industries, especially for the All Sectors button.

<IPython.core.display.Javascript object>